# LangChain: Models, Prompts and Output Parsers

## Outline

 * Direct API calls to GroqAPI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [1]:
import os
from groq import Groq
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

def get_completion(prompt, model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1 # 1 seems to give the best results
    )
    return response.choices[0].message.content


#### The code below used to select the model version at the time of code execution. Nonetheless, because I am using the GroqAPI, I have much more flexibility in choosing the LLM to promt, and the one used in these notebooks is Meta's LLaMA3 70b model with a context window of 8,192 tokens.

In [ ]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Chat API : GroqAI

Let's start with a direct API calls to Groq.

In [3]:
get_completion("What is 1+1?", model="llama3-70b-8192")


'The answer to 1+1 is 2.'

In [4]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [5]:
style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [7]:
response = get_completion(prompt, model="llama3-70b-8192")
print(response)

Here is the translated text:

```
I'm extremely frustrated that my blender lid came loose and splattered my kitchen walls with smoothie all over the place! And to make matters worse, the warranty doesn't cover the cost of cleaning up this mess. I really need your help right now, please!
```

I've translated the text to use American English and a calm and respectful tone, removing the pirate-themed language and slang to make the text more polite and professional. Let me know if you have any further requests!


In [10]:
response = get_completion("How can I use Langchain to prompt the GroqAPI?", model="mixtral-8x7b-32768")
print(response) ## Just playing around, but the llm hulucinations are quite interesting!

I'm sorry for the confusion, but it seems there might be a misunderstanding. Langchain is a platform for developers to build, train, and deploy language models, while GroqAPI is an API for accessing the Groq database and processing data using its query language.

To use GroqAPI, you don't need Langchain. You'll need to make HTTP requests (usually via a programming language such as Python, JavaScript, or Go) using the GroqAPI's endpoints. You can use the Groq query language or GQL for your data processing tasks.

Here are the basic steps to get started with GroqAPI:

1. Register for an API key at <https://auth.groq.com/register>.
2. Install the Groq CLI or a Groq language SDK for your preferred programming language (see the official Groq documentation).
3. Write your data requests using the Groq query language.
4. Use the API key and query in the request to the GroqAPI endpoints.
5. Parse and process the response.

To use Langchain, you'd follow the instructions on the platform to build

## Chat API : LangChain

Let's try how we can do the same using LangChain. To use GroqAPi, we must install 'langchain-groq' instead of 'langchain', see [Official Docs](https://python.langchain.com/v0.2/docs/integrations/chat/groq/) for refrence

In [65]:
#!pip install --upgrade langchain-groq

### Model

In [17]:
from langchain_groq import ChatGroq #Langchai abstraction for GroqAPI endpoint

chat = ChatGroq(temperature=0.0, model_name="llama3-70b-8192")

chain = prompt | chat
chain.invoke({"text": "Explain the importance of low latency LLMs."}).content

'Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:\n\n1. **Conversational AI**: In conversational AI, low-latency LLMs enable more natural and human-like interactions. Fast response times are critical in chatbots, virtual assistants, and voice assistants to maintain user engagement and provide a seamless experience.\n2. **Real-time Language Translation**: Low-latency LLMs are vital in real-time language translation applications, such as simultaneous interpretation, live subtitles, and multilingual customer support. Fast processing ensures that translations are provided quickly, enabling effective communication across languages.\n3. **Sentiment Analysis and Feedback**: In applications like customer feedback analysis, low-latency LLMs help process and analyze user feedback in real-time, enabling businesses to respond promptly to customer concerns

In [15]:
chat

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a6997de5630>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a6997de56f0>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********'))

### Prompt template

In [16]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [18]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [19]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [20]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [21]:
print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages[0])

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [22]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages).content
print(customer_response)

Here is the translated text in American English, in a calm and respectful tone:

"I'm extremely frustrated that my blender lid came loose and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now, please."


In [23]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [24]:
service_response = chat.invoke(service_messages).content
print(service_response)

Here be the rewritten text, matey:

```Ahoy, valued customer! I be afraid the warranty don't cover the cleanin' o' yer kitchen, savvy? It seems ye forgot to put the lid on yer blender afore turnin' it on, and that be a mighty big no-no, matey! Alas, that be yer responsibility, not ours. Mayhap next time ye'll be more mindful o' yer blenderin' ways. Fair winds, and may yer future blendin' adventures be lid-tastic!```
